# Pointwise Evaluation

### Imports

In [1]:
import os
import sys
from skopt.space import Integer
from skopt.space import Real
from skopt.space import Categorical
sys.path.append(os.path.dirname((os.path.abspath(""))))

In [2]:
from src.pipeline import Pipeline

[nltk_data] Downloading package punkt to /Users/tim/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/tim/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/tim/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/tim/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [3]:
pipeline = Pipeline(
    collection='data/processed/30_5000_1000_collection.pkl',
    queries='data/processed/30_5000_1000_queries.pkl',
    queries_val='data/processed/30_5000_1000_queries_val.pkl',
    queries_test='data/processed/30_5000_1000_queries_test.pkl',
    features='data/processed/30_5000_1000_features.pkl',
    qrels_val='data/processed/30_5000_1000_qrels_val.pkl',
    qrels_test='data/processed/30_5000_1000_qrels_test.pkl',
    features_test='data/processed/30_5000_1000_features_test.pkl',
    features_val='data/processed/30_5000_1000_features_val.pkl',
)

In [4]:
pipeline.features

,qID,pID,y,w2v_cosine,w2v_euclidean,w2v_manhattan,w2v_tfidf_cosine,w2v_tfidf_euclidean,w2v_tfidf_manhattan,tfidf_cosine,...,polarity_doc,subjectivity_query,polarity_query,bm25,doc_nouns,doc_adjectives,doc_verbs,query_nouns,query_adjectives,query_verbs
0,603195,7050012,1,0.972107,144.641830,1124.871630,0.938781,2.765727,22.236694,0.537439,...,0.000000,0.00,0.00,-24.655536,23,6,4,3,1,1
1,474183,325505,1,0.971866,131.960266,1033.670312,0.985675,1.360485,11.347487,0.745907,...,0.450000,0.00,0.00,-33.129796,18,9,3,4,0,0
2,320545,1751825,1,0.947701,94.900002,756.378183,0.959522,2.236971,17.352688,0.409509,...,0.500000,0.20,0.20,-16.699603,20,2,14,2,1,1
3,89798,5069949,1,0.972710,161.470459,1273.643564,0.933304,1.714253,13.493497,0.541627,...,0.066667,0.25,0.00,-27.678576,25,10,5,3,1,0
4,1054603,2869106,1,0.965680,155.648453,1216.564726,0.941391,1.799412,14.369308,0.438115,...,0.000000,0.00,0.00,-28.497519,20,9,6,2,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,128401,6127598,0,0.796978,85.670822,678.466760,0.555981,3.027138,24.841764,0.185056,...,-0.520833,0.00,0.00,-8.866170,16,6,13,2,1,0
4996,1044540,4616118,0,0.922095,157.044754,1238.354322,0.603788,2.167866,17.812756,0.140057,...,0.156250,0.00,0.00,-7.852468,25,9,16,0,0,1
4997,486146,1137390,0,0.946438,125.126984,972.330644,0.882998,4.161341,34.815641,0.314505,...,-0.100000,0.10,0.00,-15.909103,12,1,10,2,0,2
4998,532697,5161847,0,0.938939,99.808395,790.453814,0.893834,1.977307,16.122506,0.344173,...,0.284375,0.00,0.00,-16.617979,18,8,9,3,1,0


### Logistic Regression

##### Without Hyperparameter Optimization

In [5]:
pipeline.evaluate(
    name='default',
    model='lr', 
    pca=0
)

/Users/tim/opt/anaconda3/envs/inforetrieval/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


MRR: 0.03805952869994696


##### With Hyperparameter Optimization

In [5]:
logistic_regression_search_space: list = []
logistic_regression_search_space.append(Categorical(['l2', 'none'], name='penalty'))
logistic_regression_search_space.append(Real(0.1, 100.0, name='C'))
logistic_regression_search_space.append(Real(1e-6, 0.1, name='tol'))

In [6]:
pipeline.evaluate(
    name='hpo',
    model='lr', 
    pca=5, 
    search_space=logistic_regression_search_space,
    trials=20
)

/Users/tim/opt/anaconda3/envs/inforetrieval/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1483: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/tim/opt/anaconda3/envs/inforetrieval/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1483: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/tim/opt/anaconda3/envs/inforetrieval/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1483: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/tim/opt/anaconda3/envs/inforetrieval/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1483: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/tim/opt/anaconda3/envs/inforetrieval/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1483: UserWarning: Setting penalty='none' will ignore the C and l1

Best MRR: -0.020961472527053267
Best Hyperparameters: ['l2', 5.6980759407867705, 0.002508188700257655]
MRR on test set: 0.037126943017010396


### Naive Bayes

In [6]:
pipeline.evaluate(
    name='default',
    model='nb', 
    pca=0
)

MRR: 0.03607157243379598


### Multi-layer Perceptron

##### Without Hyperparameter Optimization

In [16]:
pipeline.evaluate(
    name='default',
    model='mlp', 
    pca=0
)

/Users/tim/opt/anaconda3/envs/inforetrieval/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


MRR: 0.021087456150293574


##### With Hyperparameter Optimization

In [13]:
mlp_search_space: list = []
mlp_search_space.append(Categorical(['identity', 'logistic', 'tanh', 'relu'], name='activation'))
mlp_search_space.append(Real(1e-6, 0.1, name='alpha'))
mlp_search_space.append(Real(1e-6, 0.1, name='learning_rate_init'))

In [17]:
pipeline.evaluate(
    name='hpo',
    model='mlp', 
    pca=0,
    search_space=mlp_search_space,
    trials=10
)

Best MRR: 0.008856659110087957
Best Hyperparameters: ['relu', 0.0057874644134355835, 0.0478341107632347]
MRR on test set: 0.029243423005499914


### Support Vector Machine

##### Without Hyperparameter Optimization

In [8]:
pipeline.evaluate(
    name='default',
    model='svm', 
    pca=0
)

MRR: 0.0319047903484395


##### With Hyperparameter Optimization

In [18]:
svm_search_space: list = []
svm_search_space.append(Categorical(['poly', 'rbf', 'sigmoid'], name='kernel'))
svm_search_space.append(Real(0.1, 100.0, name='C'))

In [19]:
pipeline.evaluate(
    model='svm', 
    pca=0,
    search_space=svm_search_space,
    trials=10
)

Best MRR: 0.006591189472904975
Best Hyperparameters: ['poly', 96.43409248718199]
MRR on test set: 0.02065391377002617


### Decision Tree

In [16]:
from sklearn.tree import DecisionTreeClassifier
import json

dt = DecisionTreeClassifier()
str(dt.get_params())

"{'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'random_state': None, 'splitter': 'best'}"

##### Without Hyperparameter Optimization

In [7]:
pipeline.evaluate(
    name='default',
    model='dt', 
    pca=0
)

MRR: 0.00562915260933


##### With Hyperparameter Optimization

In [6]:
decision_tree_search_space: list = []
decision_tree_search_space.append(Categorical(['gini', 'entropy'], name='criterion'))
decision_tree_search_space.append(Integer(2, 15, name='min_samples_split'))
decision_tree_search_space.append(Integer(1, 10, name='min_samples_leaf'))
decision_tree_search_space.append(Integer(5, 100, name='max_leaf_nodes'))
decision_tree_search_space.append(Integer(10, 50, name='max_depth'))
decision_tree_search_space.append(Real(0.0, 0.2, name='min_weight_fraction_leaf'))

In [7]:
pipeline.evaluate(
    model='dt', 
    pca=0,
    search_space=decision_tree_search_space
)

Best MRR: -0.013578619748480097
Best Hyperparameters: ['entropy', 8, 10, 46, 48, 0.190366916117415]
MRR on test set: 0.02660054996605312


### Random Forest

##### Without Hyperparameter Optimization

In [10]:
pipeline.evaluate(
    name='default',
    model='rf', 
    pca=0
)

MRR: 0.01832565851787843


### Adaptive Boosting

##### Without Hyperparameter Optimization

In [11]:
pipeline.evaluate(
    name='default',
    model='ada', 
    pca=0
)

MRR: 0.03265656237712727


### Gradient Boosting

##### Without Hyperparameter Optimization

In [8]:
pipeline.evaluate(
    name='default',
    model='gb', 
    pca=0
)

MRR: 0.024049391304517796


In [10]:
import pandas as pd
from src.utils.utils import load

results = load('data/results/results.pkl')

### Default Results

In [14]:
results[results['name'] == 'default']

,name,model,hyperparameters,pairwise_model,pairwise_k,features,sampling_training,sampling_test,pca,MRR,MAP,nDCG,accuracy,precision,recall,f1,accuracy@50,precision@50,recall@50,f1@50
17,default,LogisticRegression(),"{""C"": 1.0, ""class_weight"": null, ""dual"": false...",None,None,"[""w2v_cosine"", ""w2v_euclidean"", ""w2v_manhattan...",9977,451680,0,0.038060,0.865090,0.859446,0.997135,0.993724,0.268969,0.423351,0.690667,0.993289,0.490608,0.656805
18,default,GaussianNB(),"{""priors"": null, ""var_smoothing"": 1e-09}",None,None,"[""w2v_cosine"", ""w2v_euclidean"", ""w2v_manhattan...",9977,451680,0,0.036072,0.874882,0.875631,0.997173,0.995943,0.278029,0.434706,0.718000,0.995851,0.532741,0.694143
19,default,MLPClassifier(),"{""activation"": ""relu"", ""alpha"": 0.0001, ""batch...",None,None,"[""w2v_cosine"", ""w2v_euclidean"", ""w2v_manhattan...",9977,451680,0,0.013483,0.440844,0.638569,0.995590,0.399823,0.255379,0.311679,0.623333,0.480047,0.770245,0.591468
20,default,SVC(probability=True),"{""C"": 1.0, ""break_ties"": false, ""cache_size"": ...",None,None,"[""w2v_cosine"", ""w2v_euclidean"", ""w2v_manhattan...",9977,451680,0,0.031905,0.775297,0.826087,0.997208,0.994129,0.287656,0.446201,0.748667,0.993776,0.561547,0.717603
21,default,DecisionTreeClassifier(),"{""ccp_alpha"": 0.0, ""class_weight"": null, ""crit...",None,None,"[""w2v_cosine"", ""w2v_euclidean"", ""w2v_manhattan...",9977,451680,0,0.008408,0.308170,0.576857,0.996393,0.560889,0.357305,0.436527,0.716000,0.555556,0.996241,0.713324
22,default,RandomForestClassifier(),"{""bootstrap"": true, ""ccp_alpha"": 0.0, ""class_w...",None,None,"[""w2v_cosine"", ""w2v_euclidean"", ""w2v_manhattan...",9977,451680,0,0.018326,0.677792,0.820648,0.997430,0.993475,0.344847,0.511980,0.850667,0.992920,0.718310,0.833581
23,default,AdaBoostClassifier(),"{""algorithm"": ""SAMME.R"", ""base_estimator"": nul...",None,None,"[""w2v_cosine"", ""w2v_euclidean"", ""w2v_manhattan...",9977,451680,0,0.032657,0.827516,0.854278,0.997219,0.992278,0.291053,0.450088,0.718667,0.991667,0.532438,0.692868
24,default,GradientBoostingClassifier(),"{""ccp_alpha"": 0.0, ""criterion"": ""friedman_mse""...",None,None,"[""w2v_cosine"", ""w2v_euclidean"", ""w2v_manhattan...",9977,451680,0,0.024211,0.728175,0.827123,0.997250,0.992481,0.298981,0.459530,0.771333,0.991903,0.591074,0.740741


### Hyperparameter Optimization

In [11]:
results[results['name'] == 'hpo']

,name,model,hyperparameters,pairwise_model,pairwise_k,features,sampling_training,sampling_test,pca,MRR,MAP,nDCG,accuracy,precision,recall,f1,accuracy@50,precision@50,recall@50,f1@50
25,hpo,"LogisticRegression(C=0.10099145980936619, tol=...","{""C"": 0.10099145980936619, ""class_weight"": nul...",None,None,"[""pca_comp_0"", ""pca_comp_1"", ""pca_comp_2"", ""pc...",9977,451680,5,0.037135,0.882746,0.871568,0.996918,0.994709,0.212911,0.350746,0.642,0.994709,0.412733,0.583398
